# BTE metakg visualization

The goal of this notebook is to visualize the metaKG that BTE uses.  Similar to the [subway diagram](https://raw.githubusercontent.com/biothings/BioThings_Explorer_TRAPI/main/diagrams/smartapi_metagraph.png) we've used before, but updated to the size and scale of the current metakg.

This notebook takes as input an ndson file with the SmartAPI metakg (originally provided by Chunlei on 2023-03-01).


In [1]:
import pandas as pd
import requests
import json5
import re

## Read in data

First, read in the Smart API ndjson file

In [2]:
df = pd.read_json('data/smartapi_metakg_03012023.ndjson', lines=True)
df

,subject,object,predicate,api,provided_by
0,AnatomicalEntity,MolecularActivity,affects_activity_of,"{'name': 'CAM-KP API', 'smartapi': {'metadata'...",NaN
1,AnatomicalEntity,MolecularEntity,affects_activity_of,"{'name': 'CAM-KP API', 'smartapi': {'metadata'...",NaN
2,AnatomicalEntity,NamedThing,affects_activity_of,"{'name': 'CAM-KP API', 'smartapi': {'metadata'...",NaN
3,AnatomicalEntity,NucleicAcidEntity,affects_activity_of,"{'name': 'CAM-KP API', 'smartapi': {'metadata'...",NaN
4,AnatomicalEntity,Occurrent,affects_activity_of,"{'name': 'CAM-KP API', 'smartapi': {'metadata'...",NaN
...,...,...,...,...,...
175543,Cell,Phenomenon,causes,{'name': 'ARAX Translator Reasoner - TRAPI 1.3...,NaN
175544,Transcript,CellLine,physically_interacts_with,{'name': 'ARAX Translator Reasoner - TRAPI 1.3...,NaN
175545,Disease,PhenotypicFeature,entity_positively_regulates_entity,{'name': 'ARAX Translator Reasoner - TRAPI 1.3...,NaN
175546,Device,Vitamin,entity_negatively_regulates_entity,{'name': 'ARAX Translator Reasoner - TRAPI 1.3...,NaN


parse out a couple lines for the API name and ID

In [3]:
df = df.assign(api_name = lambda x: pd.json_normalize(x['api'])['name'])
df = df.assign(api_id = lambda x: pd.json_normalize(x['api'])['smartapi.id'])
df

,subject,object,predicate,api,provided_by,api_name,api_id
0,AnatomicalEntity,MolecularActivity,affects_activity_of,"{'name': 'CAM-KP API', 'smartapi': {'metadata'...",NaN,CAM-KP API,c409da43f5177ae52392e2b346dbc585
1,AnatomicalEntity,MolecularEntity,affects_activity_of,"{'name': 'CAM-KP API', 'smartapi': {'metadata'...",NaN,CAM-KP API,c409da43f5177ae52392e2b346dbc585
2,AnatomicalEntity,NamedThing,affects_activity_of,"{'name': 'CAM-KP API', 'smartapi': {'metadata'...",NaN,CAM-KP API,c409da43f5177ae52392e2b346dbc585
3,AnatomicalEntity,NucleicAcidEntity,affects_activity_of,"{'name': 'CAM-KP API', 'smartapi': {'metadata'...",NaN,CAM-KP API,c409da43f5177ae52392e2b346dbc585
4,AnatomicalEntity,Occurrent,affects_activity_of,"{'name': 'CAM-KP API', 'smartapi': {'metadata'...",NaN,CAM-KP API,c409da43f5177ae52392e2b346dbc585
...,...,...,...,...,...,...,...
175543,Cell,Phenomenon,causes,{'name': 'ARAX Translator Reasoner - TRAPI 1.3...,NaN,ARAX Translator Reasoner - TRAPI 1.3.0,e248aefca0f469229e82cca80fbabc89
175544,Transcript,CellLine,physically_interacts_with,{'name': 'ARAX Translator Reasoner - TRAPI 1.3...,NaN,ARAX Translator Reasoner - TRAPI 1.3.0,e248aefca0f469229e82cca80fbabc89
175545,Disease,PhenotypicFeature,entity_positively_regulates_entity,{'name': 'ARAX Translator Reasoner - TRAPI 1.3...,NaN,ARAX Translator Reasoner - TRAPI 1.3.0,e248aefca0f469229e82cca80fbabc89
175546,Device,Vitamin,entity_negatively_regulates_entity,{'name': 'ARAX Translator Reasoner - TRAPI 1.3...,NaN,ARAX Translator Reasoner - TRAPI 1.3.0,e248aefca0f469229e82cca80fbabc89


Read in the BTE config file that specifies currently-allowed APIs

In [4]:
bte_config_url = "https://raw.githubusercontent.com/biothings/BioThings_Explorer_TRAPI/main/src/config/apis.js"
r = requests.get(bte_config_url)
str_bte_config = r.text
#print(str_bte_config)
str_bte_config = re.sub("exports.API_LIST = ",  "", str_bte_config)                       # remove variable assignment step
str_bte_config = re.sub("\s*//.*",              "", str_bte_config)                       # remove commented lines
str_bte_config = re.sub(r'^$\n',                '', str_bte_config, flags=re.MULTILINE)   # remove blank lines
str_bte_config = re.sub(r',\s*exclude:[^\]]*]', '', str_bte_config, flags=re.MULTILINE)   # remove 'exclude' section
str_bte_config = re.sub(r';$',                  '', str_bte_config, flags=re.MULTILINE)   # remove 'exclude' section
#print(str_bte_config)

bte_config = json5.loads(str_bte_config)
bte_config

{'include': [{'id': 'd22b657426375a5295e7da8a303b9893', 'name': 'BioLink API'},
  {'id': '43af91b3d7cae43591083bff9d75c6dd', 'name': 'EBI Proteins API'},
  {'id': 'dca415f2d792976af9d642b7e73f7a41', 'name': 'LitVar API'},
  {'id': '1f277e1563fcfd124bfae2cc3c4bcdec', 'name': 'QuickGO API'},
  {'id': '1c056ffc7ed0dd1229e71c4752239465',
   'name': 'Ontology Lookup Service API'},
  {'id': '38e9e5169a72aee3659c9ddba956790d',
   'name': 'BioThings BindingDB API'},
  {'id': '55a223c6c6e0291dbd05f2faf27d16f4',
   'name': 'BioThings BioPlanet Pathway-Disease API'},
  {'id': 'b99c6dd64abcefe87dcd0a51c249ee6d',
   'name': 'BioThings BioPlanet Pathway-Gene API'},
  {'id': '00fb85fc776279163199e6c50f6ddfc6', 'name': 'BioThings DDInter API'},
  {'id': 'e3edd325c76f2992a111b43a907a4870', 'name': 'BioThings DGIdb API'},
  {'id': 'a7f784626a426d054885a5f33f17d3f8', 'name': 'BioThings DISEASES API'},
  {'id': '1f47552dabd67351d4c625adb0a10d00',
   'name': 'BioThings EBIgene2phenotype API'},
  {'id': 'cc

In [5]:
bte_config_ids = [ x['id'] for x in bte_config['include'] ]
bte_config_ids

['d22b657426375a5295e7da8a303b9893',
 '43af91b3d7cae43591083bff9d75c6dd',
 'dca415f2d792976af9d642b7e73f7a41',
 '1f277e1563fcfd124bfae2cc3c4bcdec',
 '1c056ffc7ed0dd1229e71c4752239465',
 '38e9e5169a72aee3659c9ddba956790d',
 '55a223c6c6e0291dbd05f2faf27d16f4',
 'b99c6dd64abcefe87dcd0a51c249ee6d',
 '00fb85fc776279163199e6c50f6ddfc6',
 'e3edd325c76f2992a111b43a907a4870',
 'a7f784626a426d054885a5f33f17d3f8',
 '1f47552dabd67351d4c625adb0a10d00',
 'cc857d5b7c8b7609b5bbb38ff990bfff',
 'f339b28426e7bf72028f60feefcd7465',
 '34bad236d77bea0a0ee6c6cba5be54a6',
 '316eab811fd9ef1097df98bcaa9f7361',
 'a5b0ec6bfde5008984d4b6cde402d61f',
 '32f36164fabed5d3abe6c2fd899c9418',
 '77ed27f111262d0289ed4f4071faa619',
 'edeb26858bd27d0322af93e7a9e08761',
 '03283cc2b21c077be6794e1704b1d230',
 '1d288b3a3caf75d541ffaae3aab386c8',
 'ec6d76016ef40f284359d17fbf78df20',
 '8f08d1446e0bb9c2b323713ce83e2bd3',
 '671b45c0301c8624abbd26ae78449ca2',
 '59dce17363dce279d389100834e43648',
 '09c8782d9f4027712e65b95424adba79',
 

## Join SmartAPI data with BTE config IDs

In [6]:
df_bte = df.query('api_id in @bte_config_ids')
df_bte

,subject,object,predicate,api,provided_by,api_name,api_id
3000,SequenceVariant,Gene,is_sequence_variant_of,"{'name': 'LitVar API', 'smartapi': {'metadata'...",infores:dbsnp,LitVar API,dca415f2d792976af9d642b7e73f7a41
5115,Gene,Gene,orthologous_to,"{'name': 'Automat-biolink(Trapi v1.3.0)', 'sma...",NaN,Automat-biolink(Trapi v1.3.0),25085b05fd1afcebb497724d147cfb44
5116,Gene,Gene,superclass_of,"{'name': 'Automat-biolink(Trapi v1.3.0)', 'sma...",NaN,Automat-biolink(Trapi v1.3.0),25085b05fd1afcebb497724d147cfb44
5117,Gene,Gene,biomarker_for,"{'name': 'Automat-biolink(Trapi v1.3.0)', 'sma...",NaN,Automat-biolink(Trapi v1.3.0),25085b05fd1afcebb497724d147cfb44
5118,Gene,Gene,phenotype_of,"{'name': 'Automat-biolink(Trapi v1.3.0)', 'sma...",NaN,Automat-biolink(Trapi v1.3.0),25085b05fd1afcebb497724d147cfb44
...,...,...,...,...,...,...,...
130260,PhenotypicFeature,SmallMolecule,contribution_from,{'name': 'Text Mining Targeted Association API...,NaN,Text Mining Targeted Association API,978fe380a147a8641caf72320862697b
130261,SmallMolecule,Disease,contributes_to,{'name': 'Text Mining Targeted Association API...,NaN,Text Mining Targeted Association API,978fe380a147a8641caf72320862697b
130262,Disease,SmallMolecule,contribution_from,{'name': 'Text Mining Targeted Association API...,NaN,Text Mining Targeted Association API,978fe380a147a8641caf72320862697b
130263,SmallMolecule,Disease,contributes_to,{'name': 'Text Mining Targeted Association API...,NaN,Text Mining Targeted Association API,978fe380a147a8641caf72320862697b


## Summarization

### by subject, object; count # of APIs

In [7]:
df1 = df_bte[["subject","object","api_name"]]
api_stats = df1.groupby(['subject','object'], group_keys=False)['api_name'].nunique().rename("api_count").to_frame()
api_stats['api_list'] = df1.groupby(['subject','object'], group_keys=False)['api_name'].unique().apply(list)

with pd.option_context('display.min_rows', 20,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(api_stats)


                                               api_count  \
subject       object                                       
Activity      Activity                                 1   
              Annotation                               1   
              BiologicalProcess                        1   
              ChemicalEntity                           1   
              ChemicalExposure                         1   
              ClinicalAttribute                        1   
              ClinicalIntervention                     1   
              ComplexMolecularMixture                  1   
              Device                                   1   
              Disease                                  1   
...                                                  ...   
SmallMolecule Phenomenon                               1   
              PhenotypicFeature                        9   
              PhysiologicalProcess                     3   
              Polypeptide               

In [8]:
api_stats.to_csv("results/api_stats.tsv", sep="\t")


### by subject, object; count # of predicates

In [9]:
df1 = df_bte[["subject","object","predicate"]]
predicate_stats = df1.groupby(['subject','object'], group_keys=False)['predicate'].nunique().rename("predicate_count").to_frame()
predicate_stats['predicate_list'] = df1.groupby(['subject','object'], group_keys=False)['predicate'].unique().apply(list)

with pd.option_context('display.min_rows', 20,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(predicate_stats)


                                               predicate_count  \
subject       object                                             
Activity      Activity                                       1   
              Annotation                                     1   
              BiologicalProcess                              1   
              ChemicalEntity                                 1   
              ChemicalExposure                               1   
              ClinicalAttribute                              1   
              ClinicalIntervention                           1   
              ComplexMolecularMixture                        1   
              Device                                         1   
              Disease                                        1   
...                                                        ...   
SmallMolecule Phenomenon                                     1   
              PhenotypicFeature                             16   
          

In [10]:
predicate_stats.to_csv("results/predicate_stats.tsv", sep="\t")